<a href="https://colab.research.google.com/github/quangnguyen-james/Datamining-Technical/blob/main/online_retail_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import random

In [2]:
process_data = pd.read_excel('/content/Online Retail.xlsx')
process_data.info()
#bread_data.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [3]:
#Kiểm tra duplicate
print(process_data.duplicated().sum())
# Lấy ra 10 bản ghi bị duplicate
duplicates = process_data[process_data.duplicated()].head(10)
print(duplicates)

5268
    InvoiceNo StockCode                        Description  Quantity  \
517    536409     21866        UNION JACK FLAG LUGGAGE TAG         1   
527    536409     22866      HAND WARMER SCOTTY DOG DESIGN         1   
537    536409     22900    SET 2 TEA TOWELS I LOVE LONDON          1   
539    536409     22111       SCOTTIE DOG HOT WATER BOTTLE         1   
555    536412     22327  ROUND SNACK BOXES SET OF 4 SKULLS         1   
587    536412     22273               FELTCRAFT DOLL MOLLY         1   
589    536412     22749  FELTCRAFT PRINCESS CHARLOTTE DOLL         1   
594    536412     22141     CHRISTMAS CRAFT TREE TOP ANGEL         1   
598    536412     21448          12 DAISY PEGS IN WOOD BOX         1   
600    536412     22569        FELTCRAFT CUSHION BUTTERFLY         2   

            InvoiceDate  UnitPrice  CustomerID         Country  
517 2010-12-01 11:45:00       1.25     17908.0  United Kingdom  
527 2010-12-01 11:45:00       2.10     17908.0  United Kingdom  
537 201

In [4]:
#Xóa dữ liệu có InvoiceNo là NaN/ NUll
process_data = process_data.dropna(subset=['InvoiceNo'])
process_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
# Xóa dữ liệu trùng
process_data = process_data.drop_duplicates()
#Kiểm tra trùng
print(process_data.duplicated().sum())


0


In [6]:
# Số sample còn lại
print(process_data.shape)
process_data.head(10)

(536641, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [7]:
print((process_data['Quantity'] <= 0).sum())

10587


In [8]:
#Chỉ lấy dữ liệu có quantity > 0
process_data = process_data[process_data['Quantity'] > 0]
print(process_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 526054 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    526054 non-null  object        
 1   StockCode    526054 non-null  object        
 2   Description  525462 non-null  object        
 3   Quantity     526054 non-null  int64         
 4   InvoiceDate  526054 non-null  datetime64[ns]
 5   UnitPrice    526054 non-null  float64       
 6   CustomerID   392732 non-null  float64       
 7   Country      526054 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.1+ MB
None


In [9]:
# Loại bỏ các mẫu dữ liệu bắt đầu là ký tự
# Giữ lại các dòng mà InvoiceNo bắt đầu bằng số (0-9)
# Lọc những dòng có InvoiceNo không bị NaN
invoice_notna = process_data[process_data['InvoiceNo'].notna()]

# Giữ lại các dòng bắt đầu bằng số
process_data = invoice_notna[invoice_notna['InvoiceNo'].astype(str).str.match(r'^\d')]
process_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 526051 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    526051 non-null  object        
 1   StockCode    526051 non-null  object        
 2   Description  525459 non-null  object        
 3   Quantity     526051 non-null  int64         
 4   InvoiceDate  526051 non-null  datetime64[ns]
 5   UnitPrice    526051 non-null  float64       
 6   CustomerID   392732 non-null  float64       
 7   Country      526051 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.1+ MB


In [10]:
#Đếm số loại sản phẩm (Item)
print(process_data['StockCode'].value_counts())
#Kiểm tra có bao nhiêu sản phẩm (Item)
print(process_data['StockCode'].nunique())
print(process_data['StockCode'].unique())

StockCode
85123A    2258
85099B    2112
22423     2009
47566     1700
20725     1582
          ... 
85049c       1
23617        1
90214U       1
47591b       1
23843        1
Name: count, Length: 3940, dtype: int64
3940
['85123A' 71053 '84406B' ... '90214U' '47591b' 23843]


In [11]:
# Vì với thuật toán Apriori tính toán quá lớn, nên xử lý lâu, không đủ RAM trên Colab
#Chỉ lấy 10% dữ liệu để thử nghiệm với thuật toán Apriori
process_data = process_data.sample(frac=0.1, random_state=42)
process_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52605 entries, 286763 to 537007
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    52605 non-null  object        
 1   StockCode    52605 non-null  object        
 2   Description  52553 non-null  object        
 3   Quantity     52605 non-null  int64         
 4   InvoiceDate  52605 non-null  datetime64[ns]
 5   UnitPrice    52605 non-null  float64       
 6   CustomerID   39273 non-null  float64       
 7   Country      52605 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 3.6+ MB


In [12]:
#Gộp Items trong cùng 1 giao dịch
transactions = process_data.groupby('InvoiceNo')['Description'].apply(list)

# Chuyển tất cả phần tử trong mỗi transaction thành chuỗi
transactions_str = [[str(item) for item in transaction] for transaction in transactions]
print(transactions)


InvoiceNo
536365    [CREAM CUPID HEARTS COAT HANGER, GLASS STAR FR...
536366                             [HAND WARMER UNION JACK]
536370      [INFLATABLE POLITICAL GLOBE , STARS GIFT TAPE ]
536373                         [RETRO COFFEE MUGS ASSORTED]
536375    [GLASS STAR FROSTED T-LIGHT HOLDER, SET 7 BABU...
                                ...                        
581578    [SET/10 RED POLKADOT PARTY CANDLES, PLASTERS I...
581579    [VINTAGE CHRISTMAS BUNTING, PAPER CHAIN KIT 50...
581580    [EGG FRYING PAN RED , CHRISTMAS TREE STAR DECO...
581585    [MULTI COLOUR SILVER T-LIGHT HOLDER, SMALL MED...
581587    [CHILDRENS CUTLERY CIRCUS PARADE, ALARM CLOCK ...
Name: Description, Length: 13892, dtype: object


In [13]:
# 2. Chuyển dữ liệu sang định dạng one-hot encoding
# One-hot encoding
te = TransactionEncoder()
te_ary = te.fit(transactions_str).transform(transactions_str)

# Tạo DataFrame
df = pd.DataFrame(te_ary, columns=te.columns_)

# In kết quả
df.info()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13892 entries, 0 to 13891
Columns: 3388 entries,  4 PURPLE FLOCK DINNER CANDLES to nan
dtypes: bool(3388)
memory usage: 44.9 MB
    4 PURPLE FLOCK DINNER CANDLES   50'S CHRISTMAS GIFT BAG LARGE  \
0                           False                           False   
1                           False                           False   
2                           False                           False   
3                           False                           False   
4                           False                           False   

    DOLLY GIRL BEAKER   I LOVE LONDON MINI BACKPACK   NINE DRAWER OFFICE TIDY  \
0               False                         False                     False   
1               False                         False                     False   
2               False                         False                     False   
3               False                         False                     False   
4 

In [14]:
# 3. Chạy thuật toán Apriori
frequent_itemsets = apriori(df, min_support=0.005, use_colnames=True)
print("Tập mục phổ biến:")
print(frequent_itemsets)

Tập mục phổ biến:
     support                              itemsets
0   0.008206              (6 RIBBONS RUSTIC CHARM)
1   0.005039     (60 CAKE CASES VINTAGE CHRISTMAS)
2   0.007198         (60 TEATIME FAIRY CAKE CASES)
3   0.006982          (ALARM CLOCK BAKELIKE GREEN)
4   0.008998           (ALARM CLOCK BAKELIKE RED )
..       ...                                   ...
91  0.017276  (WHITE HANGING HEART T-LIGHT HOLDER)
92  0.005183   (WOOD BLACK BOARD ANT WHITE FINISH)
93  0.006838         (WOODEN FRAME ANTIQUE WHITE )
94  0.007702   (WOODEN PICTURE FRAME WHITE FINISH)
95  0.005975              (WOODLAND CHARLOTTE BAG)

[96 rows x 2 columns]


In [15]:
# 4. Sinh luật kết hợp
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
print("\nLuật kết hợp:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


Luật kết hợp:
Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


In [ ]:
# 5. Lưu kết quả vào file CSV (tùy chọn)
rules.to_csv('association_rules_output.csv', index=False)
print("\nKết quả luật kết hợp đã được lưu vào file 'break_association_rules_output.csv'")